In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage import filters, img_as_ubyte
from skimage.io import imread, imsave, imread_collection
from skimage.transform import resize
# from ultralytics import YOLO
from tqdm import tqdm

In [ ]:
def smoothing_mask_edges(mask):
    tmp = filters.gaussian(mask, sigma=3)
    return tmp > filters.threshold_otsu(tmp)

In [ ]:
def try_all_models(image):
    
    paths = [
        'yolov8x-seg.pt',
        'yolov8l-seg.pt',
        'yolov8m-seg.pt',
        'yolov8s-seg.pt',
        'yolov8n-seg.pt',
    ]
    
    for model_path in paths:
        
        try:
            model = YOLO("models/" + model_path)

            preds, *_ = model.predict(source=image)
            
            return preds.masks.masks
        except:
            print("Model Error: ", model_path)
            continue
    return ()

In [ ]:
for index, frame in tqdm(enumerate(imread_collection("../data/coletas/combined/images/*"))):
    
    height, width, channels = frame.shape
    
    result = np.ones((512, 640))
    
    for mask in try_all_models(frame): result += np.asarray(mask)
    
    mask_resized = resize(result, (height, width), anti_aliasing=True)
    
    mask_smoothed = smoothing_mask_edges(mask_resized)
    
    imsave(f"output/{index}.png", img_as_ubyte(mask_smoothed))

In [ ]:
def split_mask_v2(mask):
    thresh = mask.copy().astype(np.uint8)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) > 20:
            hull = cv2.convexHull(contour, returnPoints=False)
            defects = cv2.convexityDefects(contour, hull)
            
            if defects is None:
                continue
            
            points = []
            dd = []
            
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                d = d / 256
                if d > 1.0:
                    dd.append(d)
                    points.append(tuple(contour[f][0]))
            
            if len(points) < 2:
                continue
            
            min_dist = np.inf
            p1, p2 = None, None
            
            for i in range(len(points)):
                for j in range(i+1, len(points)):
                    dist = np.linalg.norm(np.array(points[i]) - np.array(points[j]))
                    if dist < min_dist:
                        min_dist = dist
                        p1, p2 = points[i], points[j]
            
            cv2.line(thresh, p1, p2, [255, 0, 0], 5)
    
    return thresh

def split_mask_v1(mask):

    thresh = mask.copy().astype(np.uint8)
    (contours, hierarchy) = cv2.findContours(thresh, 2, 1)

    i = 0

    for contour in contours:
        if cv2.contourArea(contour) > 2:
            hull = cv2.convexHull(contour, returnPoints=False)
            defects = cv2.convexityDefects(contour, hull)
            if defects is None:
                continue
            points = []
            dd = []

            for i in range(defects.shape[0]):
                (s, e, f, d) = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
                d = d / 256
                dd.append(d)

            for i in range(len(dd)):
                (s, e, f, d) = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
                if dd[i] > 1.0 and dd[i] / np.max(dd) > 0.2:
                    points.append(f)

            i = i + 1
            if len(points) >= 2:
                for i in range(len(points)):
                    f1 = points[i]
                    p1 = tuple(contour[f1][0])
                    nearest = None
                    min_dist = np.inf
                    for j in range(len(points)):
                        if i != j:
                            f2 = points[j]
                            p2 = tuple(contour[f2][0])
                            dist = (p1[0] - p2[0]) * (p1[0] - p2[0]) \
                                + (p1[1] - p2[1]) * (p1[1] - p2[1])
                            if dist < min_dist:
                                min_dist = dist
                                nearest = p2

                    cv2.line(thresh, p1, nearest, [0, 0, 0], 2)
    return thresh

def split_mask_v1(mask):
    thresh = mask.copy().astype(np.uint8)
    contours, hierarchy = cv2.findContours(thresh, 2, 1)
    i = 0
    for contour in contours:
        if cv2.contourArea(contour) > 20:
            hull = cv2.convexHull(contour, returnPoints=False)
            defects = cv2.convexityDefects(contour, hull)
            if defects is None:
                continue
            points = []
            dd = []

            #
            # In this loop we gather all defect points
            # so that they can be filtered later on.
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
                d = d / 256
                dd.append(d)

            for i in range(len(dd)):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
                if dd[i] > 1.0 and dd[i] / np.max(dd) > 0.2:
                    points.append(f)

            i = i + 1
            if len(points) >= 2:
                for i in range(len(points)):
                    f1 = points[i]
                    p1 = tuple(contour[f1][0])
                    nearest = None
                    min_dist = np.inf
                    for j in range(len(points)):
                        if i != j:
                            f2 = points[j]
                            p2 = tuple(contour[f2][0])
                            dist = (p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1])
                            if dist < min_dist:
                                min_dist = dist
                                nearest = p2

                    cv2.line(thresh, p1, nearest, [0, 0, 0], 5)
    return thresh


In [ ]:
image = imread("output/0.png")

In [ ]:
plt.imshow(split_mask_v1(image),  cmap="gray")